In [1]:
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
datatype = "quality"

In [3]:
if datatype == "train":
    path = '/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv'
elif datatype == "test":
    path = '/home/hassan/lily/MLA/FDA/inpatient_simple_test.csv'
else:
    path = '/home/hassan/lily/MLA/FDA/inpatient_simple_quality.csv'
    
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,CVD,unique_admitting_specialty,unique_discharging_specialty,DOMICILIARY,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,67,1,0.18,1,0,0,0,1,1,0,...,1.00,1,0,0,0,0,0,0.00,2019-04-29 22:39:20.0,0
1,200,4,17.90,2,0,0,0,4,3,0,...,0.25,1,0,0,2,0,0,130.54,2019-01-31 22:59:02.0,1
2,291,7,79.68,2,0,3,1,3,2,0,...,2.33,0,0,0,7,0,0,-6.93,2006-07-08 13:56:24.0,1
3,330,3,17.38,1,0,1,1,2,2,0,...,3.00,1,0,0,2,0,0,993.85,2019-06-28 14:50:58.0,0
4,351,3,22.14,1,0,3,1,2,2,0,...,3.00,0,2,0,0,1,1,77.26,2023-07-10 21:33:42.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,166881,1,2.52,1,0,1,1,1,1,0,...,1.00,1,0,0,0,0,0,410.96,2001-01-09 21:07:54.0,0
628,167102,3,10.25,1,0,0,0,3,1,0,...,0.38,2,0,0,1,0,0,2026.77,2011-11-27 12:57:02.0,0
629,167404,1,11.63,1,0,1,1,1,1,0,...,1.00,0,0,0,1,0,0,0.00,2004-01-30 11:12:01.0,0
630,167917,4,10.50,1,0,1,1,4,2,0,...,1.33,3,0,0,1,0,1,1354.04,2016-08-07 11:25:36.0,0


In [4]:
if datatype == "train":
    path = '/home/bhatti/dataset/VCHAMPS/lab_results_train.csv'
elif datatype == "test":
    path = '/data/public/MLA/VCHAMPS-Test/lab_results_test.csv'
else:
    path = '/data/public/MLA/VCHAMPS-Quality/lab_results_qual.csv'

lab = pd.read_csv(path, index_col=0, usecols = ["Internalpatientid", "Age at lab test", "Lab test date"])
lab = lab.reset_index()
lab["Age at lab test"] = lab["Age at lab test"].astype(str).apply(lambda x: x[:7])

In [5]:
lab = lab.drop_duplicates()
lab

,Internalpatientid,Age at lab test,Lab test date
0,100012,53.0859,2002-08-01 21:31:49.0
18,100012,53.8025,2003-04-20 20:01:15.0
19,100012,55.2741,2004-10-09 16:52:06.0
33,100012,55.3940,2004-11-22 12:41:10.0
36,100012,55.9051,2005-05-28 08:07:28.0
...,...,...,...
1970482,51551,60.9047,2015-07-28 16:50:10.0
1970503,51551,60.9077,2015-07-29 18:44:33.0
1970505,51551,60.9092,2015-07-30 07:46:49.0
1970509,51551,60.9103,2015-07-30 17:50:06.0


In [7]:
inpatient_combine = lab.merge(inpatient, how = 'left', on = 'Internalpatientid')
inpatient_combine

,Internalpatientid,Age at lab test,Lab test date,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,CVD,unique_admitting_specialty,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,100012,53.0859,2002-08-01 21:31:49.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
1,100012,53.8025,2003-04-20 20:01:15.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
2,100012,55.2741,2004-10-09 16:52:06.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
3,100012,55.3940,2004-11-22 12:41:10.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
4,100012,55.9051,2005-05-28 08:07:28.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288648,51551,60.9047,2015-07-28 16:50:10.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0
288649,51551,60.9077,2015-07-29 18:44:33.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0
288650,51551,60.9092,2015-07-30 07:46:49.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0
288651,51551,60.9103,2015-07-30 17:50:06.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0


In [8]:
inpatient_combine2 = inpatient_combine.loc[inpatient_combine['Lab test date'] <= inpatient_combine['Discharge date']]
inpatient_combine2 

,Internalpatientid,Age at lab test,Lab test date,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,CVD,unique_admitting_specialty,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,100012,53.0859,2002-08-01 21:31:49.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
1,100012,53.8025,2003-04-20 20:01:15.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
2,100012,55.2741,2004-10-09 16:52:06.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
3,100012,55.3940,2004-11-22 12:41:10.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
4,100012,55.9051,2005-05-28 08:07:28.0,6.0,5.04,3.0,2.0,0.0,0.0,4.0,...,1.00,5.0,0.0,0.0,0.0,1.0,0.0,4130.09,2007-08-27 10:48:05.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288643,51551,54.9455,2009-08-10 14:01:27.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0
288644,51551,59.2585,2013-12-03 23:48:20.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0
288645,51551,60.8355,2015-07-03 09:34:30.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0
288646,51551,60.8359,2015-07-03 12:41:20.0,2.0,3.25,2.0,0.0,2.0,1.0,2.0,...,0.29,1.0,0.0,0.0,1.0,0.0,1.0,22.20,2015-07-04 00:57:40.0,1.0


In [9]:
lab_results_features = []
for ids, group in tqdm(inpatient_combine2.groupby("Internalpatientid")):
    group["Age at lab test"] = group["Age at lab test"].astype(float)
    count = group["Age at lab test"].nunique()
    
    min_age = group["Age at lab test"].min()
    max_age = group["Age at lab test"].max()
    
    freq = count/(math.floor(max_age - min_age) + 1)

    age_mean = group["Age at lab test"].mean()
    age_std = group["Age at lab test"].std()
    if group["Age at lab test"].nunique() == 1: age_std = 0

    df = pd.DataFrame(data={'Internalpatientid': [ids], 'lab_count': [count], 'lab_freq': [round(freq,2)],
                            "lab_age_mean": age_mean, "lab_age_std": age_std})
    
    df = df.reset_index(drop=True)
    lab_results_features.append(df)
    
lab_results_features = pd.concat(lab_results_features)

  0%|          | 0/631 [00:00<?, ?it/s]

In [10]:
if datatype == "train": lab_results_features.to_csv("/home/hassan/lily/MLA/FDA/inpatient_lab_results.csv")
elif datatype == "test": lab_results_features.to_csv("/home/hassan/lily/MLA/FDA/inpatient_lab_results_test.csv")
else: lab_results_features.to_csv("/home/hassan/lily/MLA/FDA/inpatient_lab_results_quality.csv")
lab_results_features

,Internalpatientid,lab_count,lab_freq,lab_age_mean,lab_age_std
0,67,36,4.00,52.117545,2.576269
0,200,222,12.33,77.161244,4.392966
0,291,291,72.75,82.348391,1.191165
0,330,147,7.74,66.364392,4.912354
0,351,62,62.00,84.806708,0.392140
...,...,...,...,...,...
0,166881,23,11.50,74.585437,0.461094
0,167102,172,21.50,65.819513,2.539545
0,167404,47,23.50,77.886036,0.237053
0,167917,48,9.60,44.236412,1.515158
